In [31]:
import torch

## Creating Layers

In [32]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases

## Activation Functions

### ReLU

In [33]:
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    self.output = torch.max(torch.tensor(0),inputs)

### Sigmoid

In [34]:
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(inputs*-1))

### Softmax

In [35]:
class Activation_Softmax:
  # Forward pass
  def forward(self, inputs):
    # Get unnormalized probabilities
    exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
    # Normalize them for each sample
    probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
    self.output = probabilities

###Linear Activation

In [36]:
class LinearActivation:
    def forward(self, x):
        self.output = x

In [37]:
X = torch.tensor([0.1, 0.5])
y = torch.tensor([0.05, 0.95])
print(X)

print(y)


tensor([0.1000, 0.5000])
tensor([0.0500, 0.9500])


###Use 2 features in the input layer, 1 hidden layer with 4 neurons

In [38]:
hidden_layer_1 = DenseLayer(2, 4)

###Sigmoid activation in the hidden layer

In [39]:
activation1 = Activation_Sigmoid()

###An output layer with 2 neurons.

In [40]:
output_layer = DenseLayer(4, 2)

###linear activation in the output layer.

In [41]:
activation2 = LinearActivation()

###Forward Pass

In [42]:
def forward_pass(X):
  hidden_layer_1.forward(X)
  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)
  activation2.forward(output_layer.output)
  return activation2.output

###Backward propagation

In [43]:
def back_prop(fp):
    lr = torch.tensor(0.01)
    back1 = (fp[0][0] - y[0]) * (1 - fp[0][0]) * fp[0][0]
    back2 = (fp[0][1] - y[1]) * (1 - fp[0][1]) * fp[0][1]

    output_layer.weights[0][0] -= lr * back1 * activation1.output[0][0]
    output_layer.weights[0][1] -= lr * back1 * activation1.output[0][1]
    output_layer.weights[1][0] -= lr * back2 * activation1.output[0][0]
    output_layer.weights[1][1] -= lr * back2 * activation1.output[0][1]
    output_layer.biases[0][0] -= lr * back1
    output_layer.biases[0][1] -= lr * back2

    for i in range(4):
        hidden_layer_1.weights[0][i] -= lr * (back1 * output_layer.weights[0][0] * X[0] + back2 * output_layer.weights[0][1] * X[0]) if hidden_layer_1.output[0][i] > 0 else 0
        hidden_layer_1.weights[1][i] -= lr * (back1 * output_layer.weights[1][0] * X[1] + back2 * output_layer.weights[1][1] * X[1]) if hidden_layer_1.output[0][i] > 0 else 0
        hidden_layer_1.biases[0][i] -= lr * (back1 * output_layer.weights[0][0] + back2 * output_layer.weights[0][1]) if hidden_layer_1.output[0][i] > 0 else 0


###Error Calculation(MSE)

In [44]:
def error_calculation(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)

###Do the Forward and Backward propagation

In [45]:
loss = 0.0001
y_pred = forward_pass(X)
err = error_calculation(y, y_pred)
print("Initial loss:", err)
print("Initial prediction:",y_pred)
while err > loss:
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = error_calculation(y, y_pred)
print("Final loss:", err)
print("Final prediction:",y_pred)
print("Target value:",y)

Initial loss: tensor(0.2212)
Initial prediction: tensor([[0.0090, 0.0102]])
Final loss: tensor(9.9984e-05)
Final prediction: tensor([[0.0636, 0.9354]])
Target value: tensor([0.0500, 0.9500])
